Primero se garantiza que se tengan las condiciones del ambiente de ejecución.

In [ ]:
!pip install mlflow==2.7.1
!pip install transformers[torch]==4.41.0
!pip install tqdm==4.65.0
!pip install datasets==2.14.6
!pip install scikit-learn==1.3.0
!pip install pyngrok==7.2.0
!pip install python-dotenv

In [39]:
import os
from dotenv import load_dotenv
from pyngrok import ngrok
import mlflow
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast, AdamW
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Configuración de los parámetros

Una forma sencilla de iterar los parámetros del modelo es definiéndolos como un diccionario.

In [40]:
# Define parameters

params = {
    "model_name": "distilbert-base-uncased",
    "learning_rate": 5e-5,
    "batch_size":16,
    "num_epochs": 1,
    "dataset_name": "ag_news",
    "task_name": "sequence_classification",
    "log_steps": 100,
    "max_seq_length": 128,
    "output_dir": "../models/distilbert_ag-news",
}

# Configuración de MLflow

Si vas a ejecutar el notebook usando un IDE local, debes ejecutar el siguiente comando en la terminal:

`
mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./mlruns --host 127.0.0.1 --port 5000
`

Debido a que Google Colab no permite acceso directo a `localhost`, **ngrok** crea un túnel seguro entre el entorno de Colab y una URL pública, permitiendo exponer servidores locales como el de MLflow.

In [41]:
dotenv_path = '/content/ngrok.env'
load_dotenv(dotenv_path)
authtoken = os.getenv("NGROK_AUTH_TOKEN")
ngrok.set_auth_token(authtoken)

In [42]:
# Iniciar el servidor de MLflow
get_ipython().system_raw("mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./mlruns --host 0.0.0.0 --port 5000 &")

In [43]:
# Exponer el puerto 5000 a través de ngrok
public_url = ngrok.connect("http://localhost:5000")
print(f"MLflow UI disponible en: {public_url}")

MLflow UI disponible en: NgrokTunnel: "https://b663-35-237-133-167.ngrok-free.app" -> "http://localhost:5000"


El primer paso es configurar la URI de seguimiento para MLflow

In [44]:
# mlflow.set_tracking_uri("http://localhost:5000") # Si usas el entorno local

mlflow.set_tracking_uri(public_url.public_url)

client = mlflow.MlflowClient()

experiment_name = params["task_name"]
experiment = client.get_experiment_by_name(experiment_name)

if experiment is None:

    mlflow.set_experiment(params["task_name"])
    print(f"Experimento '{experiment_name}' creado")

else:
    mlflow.set_experiment(experiment_name)
    print(f"Usando el experimento existente: '{experiment_name}")

Usando el experimento existente: 'sequence_classification


# Carga y preprocesamiento del dataset

In [45]:
# Se crea una ejecución que incluya el nombre del modelo y el dataset.
# Además se registrarán todos los parámetros del experimento
run = mlflow.start_run(run_name=f"{params['model_name']}-{params['dataset_name']}")
mlflow.log_params(params)

# Cargue del dataset
dataset = load_dataset(params['dataset_name'])

# Inicializar el tokenizador preentrenado.
# El tokenizador convierte el texto en IDs que el modelo pueda interpretar.
tokenizer = DistilBertTokenizerFast.from_pretrained(params['model_name'])

# Se define una función de tokenización para aplicar a cada lote de datos
# El padding asegura que todas las secuencias tengan la misma longitud

def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, max_length=params['max_seq_length'])

# Se aplica la tokenización por lotes: Se utilizaron muestras reducidas para agilizar el ejemplo y hacerlo más rápido.
train_dataset = dataset['train'].shuffle().select(range(500)).map(tokenize, batched=True)
test_dataset = dataset['test'].shuffle().select(range(100)).map(tokenize, batched=True)

# Crea la carpeta "data" sino existe
output_dir = "../data"
os.makedirs(output_dir, exist_ok=True)

# Guarda los dataset en la carpeta data
train_dataset.to_parquet(f"{output_dir}/train.parquet")
test_dataset.to_parquet(f'{output_dir}/test.parquet')

# Registrar los dataset como artefactos de MLflow
mlflow.log_artifact(f"{output_dir}/train.parquet", artifact_path='datasets')
mlflow.log_artifact(f"{output_dir}/test.parquet", artifact_path='datasets')

# Configurar el modelo para PyTorch y crear los DataLoaders
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

train_loader = DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=params['batch_size'], shuffle=True)

# Obtener los nombres de las etiquetas del dataset para futuros pasos
labels = dataset['train'].features['label'].names

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [46]:
mlflow.end_run()

Tener presente que los `input_ids` y `attention_mask` son columnas de los modelos de Hugging Face.


In [47]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
tokens = tokenizer("This is a sample text.", padding="max_length", truncation=True, max_length=10)

print(tokens)

{'input_ids': [101, 2023, 2003, 1037, 7099, 3793, 1012, 102, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 0, 0]}


In [ ]:
print(dataset)

print('\n')
print(dataset['train'].column_names)
print('\n')

print(dataset['train'].features)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})


['text', 'label']


{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['World', 'Sports', 'Business', 'Sci/Tech'], id=None)}


Por otra parte, el método `set_format` conivierte las columnas seleccionadas en tensores de PyTorch. <br/>

Crear `DataLoaders` para dividir los datos en batches durante el entrenamiento y la evaluación permite que el modelo procese cada grupo por separado



# Inicialización del modelo

Ahora es necesario configurar el modelo para realizar una tarea de clasificación, ajustando el modelo a las etiquetas específicas del dataset.

[main clases of pretrained models](https://huggingface.co/docs/transformers/v4.42.0/en/main_classes/model)

In [48]:
# Cargar el modelo para clasificación de secuencias
model = DistilBertForSequenceClassification.from_pretrained(params['model_name'], num_labels=len(labels))

# Mapear índices de las etiquetas con los nombres específicos de cada categoría
model.config.id2label = {i: label for i, label in enumerate(labels)}
params['id2label'] = model.config.id2label

# Aprovechar GPU en caso de que haya disponibilidad
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

# Optimización

Para minimizar el error de predicción en el dataset, el optimizador [AdamW](https://pytorch.org/docs/stable/generated/torch.optim.AdamW.html) nor servirá durante el entrenamiento. Durante el entrenamiento, el optimizador se usará junto con una función de pérdida para calcular los gradientes y realizar actualizaciones en los parámetros del modelo.

In [49]:
optimizer = AdamW(model.parameters(), lr=params['learning_rate'])

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


# Función de evaluación

Este código define una función llamada `evaluate_model` que evalúa el rendimiento de un modelo entrenado en un conjunto de datos de prueba, devolviendo varias métricas de evaluación, como **accuracy**, **precision**, **recall** y **F1**.

In [50]:
def evaluate_model(model, dataloader, device):
    model.eval() # Setea el modelo en modo evaluación, desactivando el dropout y batch normalization
    predictions, true_labels = [], [] # Listas para almacenar predicciones

    # Desactiva el cálculo de gradientes para hacer el proceso más eficiente
    with torch.no_grad():
        for batch in dataloader:
            inputs, masks, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['label'].to(device)

            # Realiza el forward pass y obtiene las predicciones del modelo
            outputs = model(inputs, attention_mask=masks)
            logits = outputs.logits  # Obtiene los logits (salidas antes de softmax)
            _, predicted_labels = torch.max(logits, dim=1)  # Selecciona la clase con el valor máximo

            # Almacena las predicciones y etiquetas verdaderas para las métricas
            predictions.extend(predicted_labels.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    # Calcula las métricas de evaluación: precisión, precisión, recall y F1
    accuracy = accuracy_score(true_labels, predictions)  # Precisión general del modelo
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='macro')

    return accuracy, precision, recall, f1

# Ciclo de entrenamiento

In [51]:
with mlflow.start_run(run_name=f"{params['model_name']}-{params['dataset_name']}") as run:

    # Registrar todos los parámetros en entrenamiento
    mlflow.log_params(params)


    with tqdm(total=params["num_epochs"] * len(train_loader), desc=f"Epoch [1/{params['num_epochs']}] - (Loss: N/A) - Steps") as pbar:

        # Iterar sobre cada época de entrenamiento
        for epoch in range(params['num_epochs']):
            running_loss = 0.0 # Variable para acumular la pérdida en cada época
            for i, batch in enumerate(train_loader, 0):
                # Pasar los datos de entrada
                inputs, masks, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['label'].to(device)

                # Reiniciar los gradientes acumulados antes del cálculo
                optimizer.zero_grad()

                # Realizar el forward pass y calcular la pérdida del modelo
                outputs = model(inputs, attention_mask=masks, labels = labels)
                loss = outputs.loss

                # Backward pass para calcular el gradiente
                loss.backward()

                # Actualizar parámetros usando el optimizador
                optimizer.step()

                # Acumular la pérdida para calcular el promedio después de un número de pasos
                running_loss += loss.item()

                # Registrar y mostrar la pérdida cada cierto número de pasos
                if i and i % params['log_steps'] == 0:
                    avg_loss = running_loss / params['log_steps'] # promedio de pérdida en los últimos log steps

                    # Actualizar la barra de progreso con el promedio de pérdida actual
                    pbar.set_description(f"Epoch [{epoch + 1}/{params['num_epochs']}] - (Loss: {avg_loss:.3f}) - Steps")

                    # Registrar la pérdida para monitoreo en MLflow
                    mlflow.log_metric("loss", avg_loss, step=epoch * len(train_loader) + i)

                    # Reiniciar la pérdida acumulada
                    running_loss = 0.0

                # Avanzar en la barra de progreso por cada lote
                pbar.update(1)

            # Evaluar el modelo en el conjunto de prueba al final de cada época
            accuracy, precision, recall, f1 = evaluate_model(model, test_loader, device)
            print(f"Epoch {epoch + 1} Metrics: Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

            # Registrar las métricas de evaluación en MLflow para monitoreo y análisis
            mlflow.log_metrics({'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1}, step=epoch)

    # Guardar el modelo en MLflow utilizando el método personalizado
    # Aquí se crea la carpeta de salida y se guardan tanto el modelo como el tokenizador
    os.makedirs(params['output_dir'], exist_ok=True)
    model.save_pretrained(params['output_dir'])
    tokenizer.save_pretrained(params['output_dir'])

    # Registrar los artefactos del modelo (pesos y configuración) en MLflow
    mlflow.log_artifacts(params['output_dir'], artifact_path="model")

    # Crear una URI única del modelo en MLflow para registrarlo y hacer referencia en el Model Registry
    model_uri = f"runs:/{run.info.run_id}/model"
    mlflow.register_model(model_uri, "agnews-transformer")

print('Finished Training')

Epoch [1/1] - (Loss: N/A) - Steps: 100%|██████████| 32/32 [05:24<00:00, 10.14s/it]

Epoch 1 Metrics: Accuracy: 0.8500, Precision: 0.8419, Recall: 0.8460, F1: 0.8365



Successfully registered model 'agnews-transformer'.
2024/10/08 22:46:54 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: agnews-transformer, version 1


Finished Training


Created version '1' of model 'agnews-transformer'.
